# This notebook collects data about all UFC bouts with detailed info from events pages

In [1]:
import requests as req
from bs4 import BeautifulSoup # this module helps in web scrapping
import pandas as pd
import random
import os 
import glob 
import re

In [4]:
url = 'https://www.tapology.com/fightcenter/promotions/1-ultimate-fighting-championship-ufc'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
data = req.get(url, headers = headers).text
soup = BeautifulSoup(data,"html.parser")

### Gets UFC events page list from tapology

In [5]:
page_list=[url]
next_page = soup.find("span", class_="next")

In [6]:
while next_page != None:
    print(next_page.a["href"])
    url = 'https://www.tapology.com/'+ str(next_page.a["href"])
    page_list.append(url)
    data = req.get(url, headers = headers).text
    soup = BeautifulSoup(data,"html.parser")
    next_page = soup.find("span", class_="next")

/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=2
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=3
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=4
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=5
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=6
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=7
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=8
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=9
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=10
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=11
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=12
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=13
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=14
/fightcenter/promotions/1-ultimate-fighting-championship-ufc?page=15
/fightcenter/promotions/1-ultimate-fightin

### Gets UFC events list from tapology

In [7]:
from tqdm import tqdm

In [21]:
events = pd.DataFrame(columns=["Event Link", "Event Name", "Date", "Venue", "Location"])
# Iterates through page_list
for page in tqdm(page_list):
    data = req.get(page, headers = headers).text
    soup = BeautifulSoup(data,"html.parser")
    # Finds all listings on a page
    listings = soup.find_all(class_="fcListing")
    # Iterates listings and saves each one in DataFrame
    for listing in listings:
        link = "https://www.tapology.com" + listing.find("a")["href"]
        name = listing.find("a").text
        date = listing.find(class_="datetime").text
        venue = ""
        if listing.find(class_="venue") != None: 
            venue = listing.find("span", class_="venue").text
        location = ""
        if listing.find(class_="venue-location") != None:     
            location = listing.find(class_="venue-location").text
        region = ""
        if listing.find(class_="region") != None: 
            region = listing.find(class_="region").text
        events = events._append({"Event Link":link, "Event Name":name, "Date":date, "Venue":venue, "Location":location}, ignore_index=True)
events = events.replace(r'\n','', regex=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:34<00:00,  1.20s/it]


In [22]:
events.to_csv("events.csv", index=False)

In [8]:
events=pd.read_csv("events.csv")
events

,Event Link,Event Name,Date,Venue,Location
0,https://www.tapology.com/fightcenter/events/11...,UFC Fight Night,"Saturday, September 28, 12:00 PM ET",Accor Arena,"Paris, IDF"
1,https://www.tapology.com/fightcenter/events/11...,UFC 306,"Saturday, September 14, 6:00 PM ET",The Sphere,"Las Vegas, NV"
2,https://www.tapology.com/fightcenter/events/11...,UFC Fight Night,"Saturday, August 03, 10:00 AM ET",Etihad Arena,"Abu Dhabi, AE"
3,https://www.tapology.com/fightcenter/events/11...,UFC Fight Night,"Saturday, July 13, 6:00 PM ET",NaN,NaN
4,https://www.tapology.com/fightcenter/events/10...,UFC 303,"Saturday, June 29, 6:00 PM ET",T-Mobile Arena,"Las Vegas, NV"
...,...,...,...,...,...
711,https://www.tapology.com/fightcenter/events/uf...,UFC 5: Return of the Beast,"Friday, April 07, 1995",Independence Arena,"Charlotte, NC"
712,https://www.tapology.com/fightcenter/events/uf...,UFC 4: Revenge of the Warriors,"Friday, December 16, 1994",Expo Center Pavilion,"Tulsa, OK"
713,https://www.tapology.com/fightcenter/events/uf...,UFC 3: The American Dream,"Friday, September 09, 1994",Grady Cole Center,"Charlotte, NC"
714,https://www.tapology.com/fightcenter/events/uf...,UFC 2: No Way Out,"Friday, March 11, 1994",Mammoth Gardens,"Denver, CO"


In [9]:
    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
    ]

### Gets UFC bouts list with details from tapology

In [10]:
from random import randint, uniform
from time import sleep

In [33]:
bouts = pd.DataFrame(columns=["Bout Link","Fighter A", "Link A", "Fighter B","Link B","Nickname A","Nickname B", 
                                     "Record A", "Record B","Odds A", "Odds B","Title A", "Title B","Weight A","Weight B", 
                                     "Age A", "Age B","Height A", "Height B","Reach A", "Reach B", "Result", "Time", 
                                     "Weightclass", "Rounds","Event Link"])
# Sleeping variables
event_count=0
pause_counter=0
pause_trigger=98

for ind in tqdm(events.index):
    # Sleep to avoid restriction
    sleep(uniform(0.01,2))
    if pause_counter==pause_trigger:
        pause_counter=0
        sleep(randint(10,30))
        pause_trigger=randint(80,100)
    pause_counter+=1
    
    # Copying event info
    event_link = events["Event Link"][ind]
    event_name = events["Event Name"][ind]
    # Getting page data
    headers={"User-Agent": user_agent_list[random.randint(0, len(user_agent_list)-1)]}
    data = req.get(event_link, headers = headers).text
    soup = BeautifulSoup(data,"html.parser")
    
    date, location, venue = "", "", ""
    # Event info
    event_info=soup.find(id="primaryDetailsContainer").find("ul")
    for row in event_info.find_all("li"):
        if row.find("span").text == "Date/Time:":
            date=row.find("span").findNext("span").text
        elif row.find("span").text == "Venue:":
            venue=row.find("span").findNext("span").text
        elif row.find("span").text == "Location:":
            location=row.find("span").findNext("span").text
    
    # Checking if event already finished
    if soup.find(id="eventPageHeader").find("span").contents[0].strip() == "Completed":
        event_count+=1
        #Selecting figtcard
        card = soup.find(id="sectionFightCard").find("ul") 
        #Selecting matches
        matches = card.find_all("li")
        #Scraping match info
        for match in matches: 
            # Fighters info
            fighter_a=match.find_all("a", class_="link-primary-red")[0].text
            fighter_b=match.find_all("a", class_="link-primary-red")[-2].text
            link_a="https://www.tapology.com" + match.find_all("a", class_="link-primary-red")[0]["href"]
            link_b="https://www.tapology.com" + match.find_all("a", class_="link-primary-red")[-2]["href"]
            # records after the match
            record_a=match.find_all("a", class_="link-primary-red")[0].parent.parent.parent.find_all("div")[-1].find_all("span")[-1].text
            record_b=match.find_all("a", class_="link-primary-red")[-2].parent.parent.parent.find_all("div")[-1].find_all("span")[-1].text
            
            # Match info
            result=match.find("div").find("div").find("div").find_all("span")[-2].text.strip()
            time=match.find("div").find("div").find("div").find_all("span")[-1].text.strip()
            bout_link="https://www.tapology.com" + match.find("a", class_=lambda x: x != 'link-primary-red')["href"]
            billing=match.find("a", class_=lambda x: x != 'link-primary-red').text
            weightclass=""
            if match.find("a", class_=lambda x: x != 'link-primary-red').findNext("div").text.strip() != "": 
                weightclass=match.find("a", class_=lambda x: x != 'link-primary-red').findNext("span").text.strip()
                rounds=match.find("a", class_=lambda x: x != 'link-primary-red').findNext("span").findNext("div").text.strip()
            else:
                rounds=match.find("a", class_=lambda x: x != 'link-primary-red').parent.parent.find_all("div")[-1].text.strip()
            
            # Details info
            table=match.find("table", id="boutComparisonTable")
            # Clearing variables for the next iteration - not all matches have info about every variable
            nickname_a, nickname_b, odds_a, odds_b, title_a, title_b, age_a, age_b, weight_a, weight_b, height_a, height_b, reach_a, reach_b = "", "", "", "", "", "", "", "", "", "", "", "", "", ""

            # Scraping the details table
            for row in table.find_all("tr"):
                if row.find("td") != None:
                    if row.find_all("td")[2].text.strip() == "Nickname":  
                        nickname_a=row.find_all("td")[0].text.strip()
                        nickname_b=row.find_all("td")[-1].text.strip()
                        #print(nickname_a, nickname_b)
                    elif row.find_all("td")[2].text.strip() == "Betting Odds":
                        odds_a=row.find_all("td")[0].contents[1].text.strip()
                        odds_b=row.find_all("td")[-1].contents[1].text.strip()
                        #print(odds_a, odds_b)
                    elif row.find_all("td")[2].text.strip() == "Title":
                        title_a=row.find_all("td")[0].contents[0].strip()
                        title_b=row.find_all("td")[-1].contents[0].strip()
                        #print(title_a, title_b)
                    elif row.find_all("td")[3].text.strip() == "Age at Fight":
                        age_a=row.find_all("td")[0].text.strip()
                        age_b=row.find_all("td")[-2].text.strip()
                        #print(age_a, age_b)
                    elif row.find_all("td")[2].text.strip() == "Weigh-In Result":
                        weight_a=row.find_all("td")[0].find("div").text.strip()
                        weight_b=row.find_all("td")[-1].find("div").text.strip()
                        #print(weight_a, weight_b)    
                    elif row.find_all("td")[2].text.strip() == "Height":
                        height_a=row.find_all("td")[0].find("div").text.strip()
                        height_b=row.find_all("td")[-1].find("div").text.strip()
                        #print(height_a, height_b)     
                    elif row.find_all("td")[2].text.strip() == "Reach":
                        reach_a=row.find_all("td")[0].find("div").text.strip()
                        reach_b=row.find_all("td")[-1].find("div").text.strip()
                        #print(reach_a, reach_b)
            # Populating DataFrame            
            bouts = bouts._append({'Bout Link':bout_link, 'Fighter A':fighter_a, 'Link A':link_a, 
                                                     'Fighter B':fighter_b, 'Link B':link_b, 'Nickname A':nickname_a,
                                                     'Nickname B':nickname_b, 'Record A':record_a, 'Record B':record_b, 
                                                     'Odds A':odds_a, 'Odds B':odds_b, 'Title A':title_a,'Title B':title_b, 
                                                     'Weight A':weight_a, 'Weight B':weight_b, 'Age A':age_a, 'Age B':age_b, 
                                                     'Height A':height_a,'Height B':height_b, 'Reach A':reach_a, 'Reach B':reach_b, 
                                                     'Result':result, 'Time':time, 'Weightclass':weightclass, 'Venue':venue,
                                                     'Rounds':rounds,'Date':date, 'Location':location,'Location':location,'Billing':billing,
                                                     "Event Name":event_name, "Event Link":event_link}, ignore_index=True)
bouts.info()


100%|████████████████████████████████████████████████████████████████████████████████| 713/713 [33:24<00:00,  2.81s/it]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7577 entries, 0 to 7576
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Bout Link    7577 non-null   object
 1   Fighter A    7577 non-null   object
 2   Link A       7577 non-null   object
 3   Fighter B    7577 non-null   object
 4   Link B       7577 non-null   object
 5   Nickname A   7577 non-null   object
 6   Nickname B   7577 non-null   object
 7   Record A     7577 non-null   object
 8   Record B     7577 non-null   object
 9   Odds A       7577 non-null   object
 10  Odds B       7577 non-null   object
 11  Title A      7577 non-null   object
 12  Title B      7577 non-null   object
 13  Weight A     7577 non-null   object
 14  Weight B     7577 non-null   object
 15  Age A        7577 non-null   object
 16  Age B        7577 non-null   object
 17  Height A     7577 non-null   object
 18  Height B     7577 non-null   object
 19  Reach A      7577 non-null 

In [34]:
bouts.to_csv("bouts.csv", index=False)

In [11]:
df = pd.read_csv("bouts.csv")
df.columns

Index(['Bout Link', 'Fighter A', 'Link A', 'Fighter B', 'Link B', 'Nickname A',
       'Nickname B', 'Record A', 'Record B', 'Odds A', 'Odds B', 'Title A',
       'Title B', 'Weight A', 'Weight B', 'Age A', 'Age B', 'Height A',
       'Height B', 'Reach A', 'Reach B', 'Result', 'Time', 'Weightclass',
       'Rounds', 'Event Link', 'Venue', 'Date', 'Location', 'Billing',
       'Event Name'],
      dtype='object')

### Checking for possible fighters with same name

In [12]:
print(df["Fighter A"].value_counts(), df["Link A"].value_counts())
print(df["Fighter B"].value_counts(), df["Link B"].value_counts())

Fighter A
Jim Miller          27
Andrei Arlovski     24
Donald Cerrone      24
Neil Magny          22
Demian Maia         22
                    ..
Danny Martinez       1
Garreth McLellan     1
Frankie Perez        1
Shane Campbell       1
Kevin Rosier         1
Name: count, Length: 1724, dtype: int64 Link A
https://www.tapology.com/fightcenter/fighters/jim-miller                            27
https://www.tapology.com/fightcenter/fighters/andrei-arlovski-the-pitbull           24
https://www.tapology.com/fightcenter/fighters/donald-cerrone-cowboy                 24
https://www.tapology.com/fightcenter/fighters/16456-aoutneil-magny                  22
https://www.tapology.com/fightcenter/fighters/demian-maia                           22
                                                                                    ..
https://www.tapology.com/fightcenter/fighters/danny-martinez-the-gremlin             1
https://www.tapology.com/fightcenter/fighters/16904-garreth-mclellan-soldier-boy 

We can see that 2 fighters in both groups probably have the same names with someone else

### Verify that there truly are 2 fighters with the same name in dataset

We know that one link cant lead to two fighters, but we have to check that two links dont lead to one fighter

In [13]:
filtered_dfA = df.groupby('Fighter A').filter(lambda x: len(x['Link A'].unique()) > 1)
filtered_dfB = df.groupby('Fighter B').filter(lambda x: len(x['Link B'].unique()) > 1)
pd.options.display.max_colwidth = 100
print(filtered_dfA[['Fighter A', "Link A"]])
print(filtered_dfB[['Fighter B', "Link B"]])

        Fighter A  \
473   Bruno Silva   
545   Bruno Silva   
1164  Bruno Silva   
1252  Bruno Silva   
1411  Bruno Silva   
1454  Bruno Silva   
1545  Bruno Silva   
2762  A. Nogueira   
3896  A. Nogueira   
5378  A. Nogueira   
5476  A. Nogueira   
5763  A. Nogueira   
5872  A. Nogueira   
6202  A. Nogueira   
6328  A. Nogueira   
6380  A. Nogueira   
6693  A. Nogueira   
6797  A. Nogueira   

                                                                                            Link A  
473                       https://www.tapology.com/fightcenter/fighters/59819-bruno-silva-blindado  
545   https://www.tapology.com/fightcenter/fighters/55253-bruno-gustavo-aparecido-da-silva-bulldog  
1164                      https://www.tapology.com/fightcenter/fighters/59819-bruno-silva-blindado  
1252                      https://www.tapology.com/fightcenter/fighters/59819-bruno-silva-blindado  
1411                      https://www.tapology.com/fightcenter/fighters/59819-bruno-silva-blind

After closer look its evident that the two groups of fighters with same name in the dataset are: 
1. The Nogueira brothers Antonio Rodrigo Nogueira and Antonio Rogerio Nogueira, 
2. Bruno "Blindao" Silva and Bruno "Bulldog" Silva.

We have to differentiate these names in the dataset. 

For Minotauro brothers we will change the value to their full names and for "Bulldog" and "Blindao" we will add their nicknames.

### Replacing Nogueira brothers names in both columns

In [14]:
# Replace Rogerio Nogueira in Fighter A column
condition = df["Link A"] == "https://www.tapology.com/fightcenter/fighters/antonio-rogerio-nogueira-minotoro"
df.loc[condition, "Fighter A"] = "Antonio Rogerio Nogueira"

In [15]:
# Replace Rogerio Nogueira in Fighter B column
condition = df["Link B"] == "https://www.tapology.com/fightcenter/fighters/antonio-rogerio-nogueira-minotoro"
df.loc[condition, "Fighter B"] = "Antonio Rogerio Nogueira"

In [16]:
# Replace Rodrigo Nogueira in Fighter A column
condition = df["Link A"] == "https://www.tapology.com/fightcenter/fighters/antonio-rodrigo-nogueira-minotauro"
df.loc[condition, "Fighter A"] = "Antonio Rodrigo Nogueira"

In [17]:
# Replace Rodrigo Nogueira in Fighter B column
condition = df["Link B"] == "https://www.tapology.com/fightcenter/fighters/antonio-rodrigo-nogueira-minotauro"
df.loc[condition, "Fighter B"] = "Antonio Rodrigo Nogueira"

### Replacing Bruno SIlva names  in both columns

In [18]:
# Replace Rogerio Nogueira in Fighter A column
condition = df["Link A"] == "https://www.tapology.com/fightcenter/fighters/59819-bruno-silva-blindado"
df.loc[condition, "Fighter A"] = "Bruno Blindao Silva"

In [19]:
# Replace Rogerio Nogueira in Fighter B column
condition = df["Link B"] == "https://www.tapology.com/fightcenter/fighters/59819-bruno-silva-blindado"
df.loc[condition, "Fighter B"] = "Bruno Blindao Silva"

In [20]:
# Replace Rodrigo Nogueira in Fighter A column
condition = df["Link A"] == "https://www.tapology.com/fightcenter/fighters/55253-bruno-gustavo-aparecido-da-silva-bulldog"
df.loc[condition, "Fighter A"] = "Bruno Bulldog Silva"

In [21]:
# Replace Rogerio Nogueira in Fighter B column
condition = df["Link B"] == "https://www.tapology.com/fightcenter/fighters/55253-bruno-gustavo-aparecido-da-silva-bulldog"
df.loc[condition, "Fighter B"] = "Bruno Bulldog Silva"

In [22]:
print(df["Fighter A"].value_counts(), df["Link A"].value_counts())
print(df["Fighter B"].value_counts(), df["Link B"].value_counts())

Fighter A
Jim Miller          27
Andrei Arlovski     24
Donald Cerrone      24
Neil Magny          22
Demian Maia         22
                    ..
Danny Martinez       1
Garreth McLellan     1
Frankie Perez        1
Shane Campbell       1
Kevin Rosier         1
Name: count, Length: 1726, dtype: int64 Link A
https://www.tapology.com/fightcenter/fighters/jim-miller                            27
https://www.tapology.com/fightcenter/fighters/andrei-arlovski-the-pitbull           24
https://www.tapology.com/fightcenter/fighters/donald-cerrone-cowboy                 24
https://www.tapology.com/fightcenter/fighters/16456-aoutneil-magny                  22
https://www.tapology.com/fightcenter/fighters/demian-maia                           22
                                                                                    ..
https://www.tapology.com/fightcenter/fighters/danny-martinez-the-gremlin             1
https://www.tapology.com/fightcenter/fighters/16904-garreth-mclellan-soldier-boy 

We can see that value counts in links and names are now same for fighters A and B

In [23]:
pd.concat([df["Fighter A"], df["Fighter B"]]).value_counts()

Jim Miller            43
Andrei Arlovski       41
Donald Cerrone        38
Clay Guida            36
Rafael dos Anjos      35
                      ..
John Teixeira          1
Delson Heleno          1
Anistavio Medeiros     1
Mark Hughes            1
Teila Tuli             1
Name: count, Length: 2479, dtype: int64

There are 2479 unique fighters in the dataset, now we have to connect more info from sherdog.com to them.

In [67]:
df.to_csv("bouts_new.csv", index=False)

In [24]:
df[df["Fighter B"] == "Razak Al-Hassan"]

,Bout Link,Fighter A,Link A,Fighter B,Link B,Nickname A,Nickname B,Record A,Record B,Odds A,...,Result,Time,Weightclass,Rounds,Event Link,Venue,Date,Location,Billing,Event Name
6355,https://www.tapology.com/fightcenter/bouts/362-ufc-104-kyle-kingsbu-kingsbury-vs-razak-razor-al-...,Kyle Kingsbury,https://www.tapology.com/fightcenter/fighters/kyle-kingsbury,Razak Al-Hassan,https://www.tapology.com/fightcenter/fighters/razak-al-hassan-razor,NaN,NaN,8-2,7-2,-128 (Slight Favorite),...,"Decision, Split","3 Rounds, 15:00 Total",205.0,3 x 5,https://www.tapology.com/fightcenter/events/ufc-104-machida-vs-shogun,Staples Center,Saturday 10.24.2009,"Los Angeles, California, United States",Prelim,UFC 104: Machida vs Shogun
6541,https://www.tapology.com/fightcenter/bouts/142-ufc-fight-night-16-steve-the-robot-cantwell-vs-ra...,Steve Cantwell,https://www.tapology.com/fightcenter/fighters/steve-cantwell-the-robot,Razak Al-Hassan,https://www.tapology.com/fightcenter/fighters/razak-al-hassan-razor,NaN,NaN,7-1,7-1,-318 (Moderate Favorite),...,"Submission, Armbar",4:04 Round 1 of 3,205.0,3 x 5,https://www.tapology.com/fightcenter/events/ufc-fight-night-16-fight-for-the-troops,Crown Coliseum,Wednesday 12.10.2008,"Fayetteville, North Carolina, United States",Main Card,UFC Fight Night 16: Fight for the Troops
